In [13]:
import numpy as np
import math
from check import check

In [15]:
# 状态特征
def unigram_template(off):
    def feature(seq, pos, current_tag, prev_tag):
        idx = pos + off
        if 0 <= idx < len(seq):
            return hash(seq[idx])
        else:
            return 0
    return feature

# U00, U01, U02, U03, U04
feature_functions = [
    unigram_template(-2),
    unigram_template(-1),
    unigram_template(0),
    unigram_template(1),
    unigram_template(2),
]
is_transition_feature = [False] * len(feature_functions)

In [16]:
# 组合特征
def unigram_pair_template(off1, off2):
    def feature(seq, pos, current_tag, prev_tag):
        idx1 = pos + off1
        idx2 = pos + off2
        if 0 <= idx1 < len(seq) and 0 <= idx2 < len(seq):
            return hash((seq[idx1], seq[idx2]))
        else:
            return 0
    return feature

# U05, U06, U07, U08, U09
feature_functions += [
    unigram_pair_template(-2, -1),
    unigram_pair_template(-1, 0),
    unigram_pair_template(-1, 1),
    unigram_pair_template(0, 1),
    unigram_pair_template(1, 2),
]
is_transition_feature += [False] * 5

In [17]:
# 转移特征
def bigram_template(off):
    def feature(seq, pos, current_tag, prev_tag):
        idx = pos + off
        if 0 <= idx < len(seq) and prev_tag is not None:
            return hash((seq[idx], prev_tag, current_tag))
        else:
            return 0
    return feature

# B00, B01, ..., B04
feature_functions += [
    bigram_template(-2),
    bigram_template(-1),
    bigram_template(0),
    bigram_template(1),
    bigram_template(2),
]
is_transition_feature += [True] * 5

In [18]:
def logsumexp(arr):
    max_val = np.max(arr)
    return max_val + np.log(np.sum(np.exp(arr - max_val)))

In [ ]:
class CRF:
    # n_tags: 标签数量
    # feature_functions: 特征函数列表
    def __init__(self, n_tags, feature_functions, is_transition_feature):
        self.n_tags = n_tags
        self.feature_functions = feature_functions
        self.is_transition_feature = is_transition_feature
        self.weights = np.zeros(len(feature_functions)) # 权重向量，每个特征函数对应一个权重

    # 保存模型参数
    def save_params(self, filepath):
        np.save(filepath, self.weights)
        print(f"CRF weights saved to {filepath}")

    # 加载模型参数
    def load_params(self, filepath):
        self.weights = np.load(filepath)
        print(f"CRF weights loaded from {filepath}")
    
    # 前向算法
    # alpha[t, tag]: 给定观测序列的前t+1个词时，以tag作为第t个词的标签的所有路径的“分数”之和
    def forward(self, seq):
        T = len(seq)
        alpha = np.full((T, self.n_tags), -np.inf)

        # 初始化
        for tag in range(self.n_tags):
            alpha[0, tag] = self.state_score(seq, 0, tag, None)
        
        # 递推
        for t in range(1, T):
            for tag in range(self.n_tags):
                scores = []
                for prev_tag in range(self.n_tags):
                    score = alpha[t-1, prev_tag] + self.transition_score(seq, t, tag, prev_tag)
                    scores.append(score)
                alpha[t, tag] = logsumexp(scores) + self.state_score(seq, t, tag, None)
        return alpha

    # 后向算法
    # beta[t, tag]: 在给定观测序列的第t个词处，已知该词的标签为tag，从t到序列末尾的所有可能标签路径的“分数”之和。
    def backward(self, seq):
        T = len(seq)
        beta = np.full((T, self.n_tags), -np.inf)
        
        # 初始化
        beta[T-1, :] = 0  # log(1) = 0

        # 递推
        for t in range(T-2, -1, -1):
            for tag in range(self.n_tags):
                scores = []
                for next_tag in range(self.n_tags):
                    score = (
                        beta[t+1, next_tag]
                        + self.transition_score(seq, t+1, next_tag, tag)
                        + self.state_score(seq, t+1, next_tag, tag)
                    )
                    scores.append(score)
                beta[t, tag] = logsumexp(scores)
        return beta

    # 计算边缘概率
    # marginals[t, tag]: 在给定观测序列x的条件下，第t个词的标签为tag的概率
    def compute_marginals(self, seq):
        alpha = self.forward(seq)
        beta = self.backward(seq)
        logZ = logsumexp(alpha[-1, :])
        
        marginals = np.zeros((len(seq), self.n_tags))
        for t in range(len(seq)):
            for tag in range(self.n_tags):
                marginals[t, tag] = np.exp(alpha[t, tag] + beta[t, tag] - logZ)
        return marginals
    
    # 维特比解码
    def viterbi_decode(self, seq):
        T = len(seq)
        viterbi = np.full((T, self.n_tags), -np.inf)
        backptrs = np.zeros((T, self.n_tags), dtype=int)    # 回溯指针

        # 初始化
        for tag in range(self.n_tags):
            viterbi[0, tag] = self.state_score(seq, 0, tag, None)

        # 递推
        for t in range(1, T):
            for tag in range(self.n_tags):
                max_score = -np.inf
                best_prev_tag = 0
                for prev_tag in range(self.n_tags):
                    score = viterbi[t-1, prev_tag] + self.transition_score(seq, t, tag, prev_tag)
                    if score > max_score:
                        max_score = score
                        best_prev_tag = prev_tag
                viterbi[t, tag] = max_score + self.state_score(seq, t, tag, best_prev_tag)
                backptrs[t, tag] = best_prev_tag

        # 回溯
        best_path = []
        best_last_tag = np.argmax(viterbi[-1, :])
        best_path.append(best_last_tag)

        for t in range(T-1, 0, -1):
            best_last_tag = backptrs[t, best_last_tag]
            best_path.insert(0, best_last_tag)

        return best_path
    

    # 计算状态特征得分 (直接返回log分数)
    def state_score(self, sequence, position, current_tag, prev_tag):
        score = 0
        for i, func in enumerate(self.feature_functions):
            if not self.is_transition_feature[i]:
                score += self.weights[i] * func(sequence, position, current_tag, prev_tag)
        return score

    # 计算转移特征得分 (直接返回log分数)
    def transition_score(self, sequence, position, current_tag, prev_tag):
        score = 0
        for i, func in enumerate(self.feature_functions):
            if self.is_transition_feature[i]:
                score += self.weights[i] * func(sequence, position, current_tag, prev_tag)
        return score


    # 训练CRF模型
    def train(self, seqs, tag_seqs, lr=0.01, max_iter=100):
        for iteration in range(max_iter):
            total_loss = 0
            grad = np.zeros(len(self.weights))

            for seq, tag_seq in zip(seqs, tag_seqs):
                # 计算模型预测的边缘概率
                marginals = self.compute_marginals(seq)

                empirical_expectation = np.zeros(len(self.weights))
                model_expectation = np.zeros(len(self.weights))

                # 计算经验特征期望
                # empirical_expectation[i]: 第i个特征函数在真实标签下的总和
                for t in range(len(seq)):
                    current_tag = tag_seq[t]
                    prev_tag = tag_seq[t-1] if t > 0 else None
                    for i, func in enumerate(self.feature_functions):
                        empirical_expectation[i] += func(seq, t, current_tag, prev_tag)

                # 计算模型特征期望
                # model_expectation[i]: 第i个特征函数在模型预测分布下的期望值
                for t in range(len(seq)):
                    for current_tag in range(self.n_tags):
                        for prev_tag in range(self.n_tags):
                            if t == 0:
                                prev_tag = None
                            for i, func in enumerate(self.feature_functions):
                                model_expectation[i] += marginals[t, current_tag] * \
                                                       func(seq, t, current_tag, prev_tag)

                # 更新梯度
                grad += empirical_expectation - model_expectation

                # 计算对数似然损失
                alpha = self.forward(seq)
                logZ = logsumexp(alpha[-1, :])
                sequence_score = 0
                for t in range(len(seq)):
                    current_tag = tag_seq[t]
                    prev_tag = tag_seq[t-1] if t > 0 else None
                    for i, func in enumerate(self.feature_functions):
                        sequence_score += self.weights[i] * func(seq, t, current_tag, prev_tag)
                total_loss += sequence_score - logZ

            # 更新权重
            self.weights += lr * grad

            print(f"Iteration {iteration + 1}, Loss: {total_loss}")

In [20]:
def process_data(file_path):
    # 初始化数据结构
    tags = set()
    words = set()
    tag_seqs = []
    word_seqs = []
    
    current_state_seq = []
    current_obs_seq = []
    
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line:  # 空行表示句子结束
                if current_state_seq and current_obs_seq:
                    tag_seqs.append(current_state_seq)
                    word_seqs.append(current_obs_seq)
                    current_state_seq = []
                    current_obs_seq = []
                continue
                
            parts = line.split()
            if len(parts) >= 2:  # 确保有词和标签
                word = parts[0]
                tag = parts[-1]  # 假设标签在最后
                
                # 更新状态和观测集合
                tags.add(tag)
                words.add(word)
                
                # 添加到当前序列
                current_state_seq.append(tag)
                current_obs_seq.append(word)
    
    # 处理最后一个句子（如果文件不以空行结尾）
    if current_state_seq and current_obs_seq:
        tag_seqs.append(current_state_seq)
        word_seqs.append(current_obs_seq)
    
    # 转换为列表并排序（为了确定性）
    tags = sorted(tags)
    words = sorted(words)
    
    return {
        'tags': tags,
        'words': words,
        'tag_seqs': tag_seqs,
        'word_seqs': word_seqs
    }

In [21]:
# 加载训练数据
train_data_path = "./NER/English/train.txt"
train_data = process_data(train_data_path)

tags = train_data['tags']
words = train_data['words']
tag_seqs = train_data['tag_seqs']
word_seqs = train_data['word_seqs']

In [22]:
crf = CRF(len(tags), feature_functions, is_transition_feature)

# 训练CRF模型
crf.train(word_seqs, tag_seqs, lr=0.01, max_iter=10)

Iteration 1, Loss: -447401.06566181174
Iteration 2, Loss: -6.47036864634497e+45
Iteration 3, Loss: nan


KeyboardInterrupt: 

In [23]:
valid_data_path = "./NER/English/validation.txt"
valid_data = process_data(valid_data_path)

output_path = "./crf_validation_output.txt"
with open(output_path, "w", encoding="utf-8") as fout:
    for seq, tag_seq in zip(valid_data['word_seqs'], valid_data['tag_seqs']):
        predicted_tags = crf.viterbi_decode(seq)
        for word, tag in zip(seq, predicted_tags):
            fout.write(f"{word} {tags[tag]}\n")
        fout.write("\n")  # 句子间空行

In [24]:
check(language = "English", gold_path="NER/English/validation.txt", my_path="crf_validation_output.txt")

              precision    recall  f1-score   support

       B-PER     0.0000    0.0000    0.0000      1842
       I-PER     0.0000    0.0000    0.0000      1307
       B-ORG     0.0000    0.0000    0.0000      1341
       I-ORG     0.0000    0.0000    0.0000       751
       B-LOC     0.0358    1.0000    0.0691      1837
       I-LOC     0.0000    0.0000    0.0000       257
      B-MISC     0.0000    0.0000    0.0000       922
      I-MISC     0.0000    0.0000    0.0000       346

   micro avg     0.0358    0.2135    0.0613      8603
   macro avg     0.0045    0.1250    0.0086      8603
weighted avg     0.0076    0.2135    0.0147      8603

